In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
data_folder = "../data/"

# Data loading and cleaning

## Users info

In [37]:
users      = pd.read_csv(f"{data_folder}users.csv")
users_info = pd.read_csv(f"{data_folder}users_info.csv",sep=";").set_index('attribute')

In [38]:
users_info

,type,description
attribute,,
user_id,numeric,user id
archived,ordinal,"boolean: if a user is archived, is no more vis..."
user_type,mixed,types of the user in csv
classes,categorical,classes to which the user belongs
n_activities,interval,the number of user activities
n_recipes,interval,the number of user recipes
n_experiences,interval,the number of user experiences
n_reflections,interval,the number of user reflections
n_in_curriculum,interval,the number of total activities in curriculum


**Below the list of the columns and a short description**

In the df are present those type of users:

In [15]:
print(users['user_type'].str.split(',', expand=True)[0].unique())

['formatore' 'docente' 'convocatore' 'studente' 'HGF']


We need only formatore (supervisor), docente (teacher) and studente (student)

In [16]:
# a little translation from italian
vals_to_replace = {'formatore':'supervisor', 'docente':'teacher', 'studente':'student'}
users.replace({"user_type":vals_to_replace}, inplace=True)

In [17]:
# One hot for user_type
users = pd.concat(
                [users.loc[:, :'archived'], 
                 (users['user_type'].str.split('\s*,\s*', expand=True)
                   .stack()
                   .str.get_dummies()
                   .sum(level=0)), 
                 users.loc[:, 'classes':]], 
                axis=1)

In [18]:
# remove NaN to empty classes
users.classes = users.classes.fillna('')

# fill with 0 the others
users.fillna(0,inplace=True)

## Activities info

In [32]:
activities      = pd.read_csv(f"{data_folder}activities.csv")
activities_info = pd.read_csv(f"{data_folder}activities_info.csv", sep=";").set_index("attribute")

In [33]:
activities_info

,type,description
attribute,,
activity_id,numeric,activity id
user_id,numeric,user id
at_activityType,categorical,"type of the activity ['recipe','experience']"
len_description,interval,the length of the description
ac_atSchool,ordinal,"boolean, if the recipe was done at school"
ac_atInteraziendale,ordinal,"boolean, if the recipe was done in an ""special..."
ac_atCompany,ordinal,"boolean, the recipe was done in the company"
avg_step_len,interval,average step length
n_steps,interval,the number of steps of the activity


------

# Data exploring